In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('train.csv',index_col=0)
df.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 4)

In [4]:
df.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [7]:
df.reset_index(inplace=True)

In [8]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

## Text Preprocessing

In [9]:
messages=X.copy() #deep copy

In [10]:
messages['text']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
18280    Rapper T. I. unloaded on black celebrities who...
18281    When the Green Bay Packers lost to the Washing...
18282    The Macy’s of today grew from the union of sev...
18283    NATO, Russia To Hold Parallel Exercises In Bal...
18284      David Swanson is an author, activist, journa...
Name: text, Length: 18285, dtype: object

In [11]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anju9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
lem=WordNetLemmatizer()
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]', ' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[lem.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [14]:
corpus[:5]

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery']

## One hot representation

In [15]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout
from tensorflow.keras.models import Sequential

In [16]:
voc_size=5000
one_hot_repr=[one_hot(words,voc_size) for words in corpus]
one_hot_repr[:5]

[[1636, 863, 1908, 1494, 4956, 2665, 992, 1619, 1818, 1743],
 [4054, 3937, 3878, 2845, 1068, 1816, 3228],
 [545, 3700, 4074, 3278],
 [2986, 2929, 4621, 3087, 4291, 4815],
 [4421, 1068, 1248, 1018, 2859, 1288, 1068, 2722, 480, 2685]]

## Embedding Representation

In [17]:
sent_len=20
embedded_doc=pad_sequences(one_hot_repr,padding='pre',maxlen=sent_len)
embedded_doc

array([[   0,    0,    0, ..., 1619, 1818, 1743],
       [   0,    0,    0, ..., 1068, 1816, 3228],
       [   0,    0,    0, ..., 3700, 4074, 3278],
       ...,
       [   0,    0,    0, ..., 3666,  573, 4535],
       [   0,    0,    0, ..., 2316, 4641, 2236],
       [   0,    0,    0, ..., 1930, 4873, 2249]])

In [18]:
embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1636,
        863, 1908, 1494, 4956, 2665,  992, 1619, 1818, 1743])

## Creating Model

In [19]:
dim=40
model=Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [20]:
len(embedded_doc),y.shape

(18285, (18285,))

In [21]:
X_final=np.array(embedded_doc)
y_final=np.array(y)

In [22]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.2,random_state=42)

In [24]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
229/229 [==============================] - 12s 11ms/step - loss: 0.3294 - accuracy: 0.8375 - val_loss: 0.2037 - val_accuracy: 0.9117
Epoch 2/20
229/229 [==============================] - 2s 9ms/step - loss: 0.1485 - accuracy: 0.9411 - val_loss: 0.1977 - val_accuracy: 0.9215
Epoch 3/20
229/229 [==============================] - 2s 9ms/step - loss: 0.1085 - accuracy: 0.9606 - val_loss: 0.2176 - val_accuracy: 0.9177
Epoch 4/20
229/229 [==============================] - 2s 9ms/step - loss: 0.0762 - accuracy: 0.9725 - val_loss: 0.2251 - val_accuracy: 0.9196
Epoch 5/20
229/229 [==============================] - 2s 9ms/step - loss: 0.0545 - accuracy: 0.9820 - val_loss: 0.2612 - val_accuracy: 0.9180
Epoch 6/20
229/229 [==============================] - 2s 9ms/step - loss: 0.0402 - accuracy: 0.9868 - val_loss: 0.3129 - val_accuracy: 0.9171
Epoch 7/20
229/229 [==============================] - 2s 9ms/step - loss: 0.0315 - accuracy: 0.9893 - val_loss: 0.3486 - val_accuracy: 0.9185
Epoc

In [25]:
y_pred=model.predict(X_test)

115/115 [==============================] - 1s 3ms/step


In [26]:
y_pred.round()

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [1.]], dtype=float32)

In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred.round())

array([[1881,  201],
       [ 117, 1458]], dtype=int64)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred.round())

0.9130434782608695